Data set chosen: 
   - Baseball
    
Questions posed:
   - Are salary increases correlated with better batting in the regular season over time?
   - What is the best college for baseball in the last 20 years as guaged by the number of unique players with an AllStar appearance?


In [62]:
# First I'll import the libraries that I know I will want to use
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Let's start with tracking salary over time for players as compared to their performance on the field. I'll start with tracking salaries for each player. I want to first see if anything funky is going on with the data.

In [63]:
salaries=pd.read_csv('Salaries.csv')
print salaries.describe()['salary']


count    2.557500e+04
mean     2.008563e+06
std      3.315706e+06
min      0.000000e+00
25%      2.750000e+05
50%      5.500000e+05
75%      2.250000e+06
max      3.300000e+07
Name: salary, dtype: float64


In [64]:
# I see there is a minimum salary with 0, that seems odd so I am going to remove them
non_zero_salaries=salaries[salaries.salary > 0]
print non_zero_salaries.describe()['salary']

count    2.557300e+04
mean     2.008720e+06
std      3.315788e+06
min      1.090000e+04
25%      2.750000e+05
50%      5.500000e+05
75%      2.250000e+06
max      3.300000e+07
Name: salary, dtype: float64


Now, I'll create a data set that shows me a players batting performance over time. I'll open the data, check it out to see if anything is funky and then group it up.

In [65]:
#Let's start by getting the batting regular season files

# batting=pd.read_csv('Batting.csv',header=None, 
#                     names=('playerID','yearID','stint','teamID','lgID','G','AB','R','H','Doubles',
#                            'Triples','HR','RBI','SB','CS','BB','SO','IBB','HBP','SH','SF','GIDP'),
#                     dtype={'playerID':object,'yearID':int,'stint':int,'teamID':object,'lgID':object,'G':int,\
#                            'AB':int,'R':int,'H':int,'Doubles':int,'Triples':int,'HR':int,'RBI':int,'SB':int,\
#                            'CS':int,'BB':int,'SO':int,'IBB':object,'HBP':object,'SH':object,'SF':object,'GIDP':object})
batting=pd.read_csv('Batting.csv')                   
# Take a look at a few rows to get a feel
print batting.head()

    playerID  yearID  stint teamID lgID   G     AB     R     H    2B  ...   \
0  abercda01    1871      1    TRO  NaN   1    4.0   0.0   0.0   0.0  ...    
1   addybo01    1871      1    RC1  NaN  25  118.0  30.0  32.0   6.0  ...    
2  allisar01    1871      1    CL1  NaN  29  137.0  28.0  40.0   4.0  ...    
3  allisdo01    1871      1    WS3  NaN  27  133.0  28.0  44.0  10.0  ...    
4  ansonca01    1871      1    RC1  NaN  25  120.0  29.0  39.0  11.0  ...    

    RBI   SB   CS   BB   SO  IBB  HBP  SH  SF  GIDP  
0   0.0  0.0  0.0  0.0  0.0  NaN  NaN NaN NaN   NaN  
1  13.0  8.0  1.0  4.0  0.0  NaN  NaN NaN NaN   NaN  
2  19.0  3.0  1.0  2.0  5.0  NaN  NaN NaN NaN   NaN  
3  27.0  1.0  1.0  0.0  2.0  NaN  NaN NaN NaN   NaN  
4  16.0  6.0  2.0  2.0  1.0  NaN  NaN NaN NaN   NaN  

[5 rows x 22 columns]


In [66]:
#First thing I notice is quite a few NaNs that I'll need to be aware of, particularly in the league ID, which seems odd
#I want to see just how many rows overalls have NaN for their lgID
print batting.groupby(['lgID']).count()['playerID']

lgID
AA     1890
AL    46371
FL      470
NL    51385
PL      147
UA      332
Name: playerID, dtype: int64


In [67]:
#Turns out they look ok, so I'll keep moving along to see if there 
#are any oddities amongst my columns which reflect performance
print batting.describe()['G']

count    101332.000000
mean         51.400111
std          47.145273
min           0.000000
25%          13.000000
50%          34.000000
75%          80.250000
max         165.000000
Name: G, dtype: float64


In [68]:
#I'll next remove any players with 0 games played
non_zero_batting=batting[batting.G > 0]
print non_zero_batting.groupby(['yearID']).describe()['G']

yearID       
1871    count     115.000000
        mean       19.965217
        std        10.852221
        min         1.000000
        25%        12.000000
        50%        25.000000
        75%        29.000000
        max        33.000000
1872    count     156.000000
        mean       21.198718
        std        17.491166
        min         1.000000
        25%         7.000000
        50%        17.000000
        75%        37.000000
        max        56.000000
1873    count     125.000000
        mean       28.824000
        std        21.694382
        min         1.000000
        25%         5.000000
        50%        30.000000
        75%        51.000000
        max        60.000000
1874    count     123.000000
        mean       34.138211
        std        23.981661
        min         1.000000
        25%         8.000000
        50%        40.000000
                    ...     
2012    std        44.599806
        min         1.000000
        25%        14.000000


While I don't know much about baseball, a max games played of 165 seemed high to me
After some googling I found out that there are 162 games per season, I do not see a max higher than 162/yr 
Now, I'm going to start making my definition of "good performance":

    I want to measure personal performance, so I will only look at hits+doubles+triples+homers+stolen bases in a positive light and I will look at strikeouts + caught stealing in a negative light

In [69]:
print non_zero_batting.iloc[:, 10].head()

0    0.0
1    0.0
2    5.0
3    2.0
4    3.0
Name: 3B, dtype: float64


In [70]:
#I will use iloc for the columns which have numbers in the names


non_zero_batting['performance']=pd.Series(( (non_zero_batting.H)+\
                                        (non_zero_batting.iloc[:,9])+(non_zero_batting.iloc[:, 10])\
                                        +(non_zero_batting.HR)-(non_zero_batting.CS)-(non_zero_batting.SO)))
print non_zero_batting.head()



    playerID  yearID  stint teamID lgID   G     AB     R     H    2B  \
0  abercda01    1871      1    TRO  NaN   1    4.0   0.0   0.0   0.0   
1   addybo01    1871      1    RC1  NaN  25  118.0  30.0  32.0   6.0   
2  allisar01    1871      1    CL1  NaN  29  137.0  28.0  40.0   4.0   
3  allisdo01    1871      1    WS3  NaN  27  133.0  28.0  44.0  10.0   
4  ansonca01    1871      1    RC1  NaN  25  120.0  29.0  39.0  11.0   

      ...        SB   CS   BB   SO  IBB  HBP  SH  SF  GIDP  performance  
0     ...       0.0  0.0  0.0  0.0  NaN  NaN NaN NaN   NaN          0.0  
1     ...       8.0  1.0  4.0  0.0  NaN  NaN NaN NaN   NaN         37.0  
2     ...       3.0  1.0  2.0  5.0  NaN  NaN NaN NaN   NaN         43.0  
3     ...       1.0  1.0  0.0  2.0  NaN  NaN NaN NaN   NaN         55.0  
4     ...       6.0  2.0  2.0  1.0  NaN  NaN NaN NaN   NaN         50.0  

[5 rows x 23 columns]


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


This warning is likely due to me filtering out when games were equal to 0 and is a false positive
Now I'll standardize performance across each league per year
Steps to take:
    1. Find average and std dev for each year in each league
    2. Take each individuals performance and subtract the average then divide by the std dev

In [71]:
yearly_perf_avg=pd.DataFrame(non_zero_batting.groupby(['yearID','lgID']).mean()['performance']).reset_index()

yearly_perf_dev=pd.DataFrame(non_zero_batting.groupby(['yearID','lgID']).std()['performance']).reset_index()

yearly_perf_avg.columns=['yearID','lgID','avg']
yearly_perf_dev.columns=['yearID','lgID','dev']

print yearly_perf_avg.head()
print yearly_perf_dev.head()


   yearID lgID   avg
0    1876   NL   NaN
1    1877   NL  20.0
2    1878   NL   NaN
3    1879   NL   NaN
4    1880   NL   NaN
   yearID lgID  dev
0    1876   NL  NaN
1    1877   NL  NaN
2    1878   NL  NaN
3    1879   NL  NaN
4    1880   NL  NaN


In printing my averages and devs, I notice  spotty coverage of my created performance column prior to about 2001 in these previewed years. I know there is data from before 2001 just from printing head() earlier in the project, but for simplifying my future, I'll go ahead and chop it down to the last ten years. 

In [72]:
recent_non_zero_batting=non_zero_batting[non_zero_batting.yearID>2004].reset_index()
print recent_non_zero_batting.head()

   index   playerID  yearID  stint teamID lgID    G     AB      R      H  \
0  85983  abernbr01    2005      1    MIN   AL   24   67.0    5.0   16.0   
1  85984  abreubo01    2005      1    PHI   NL  162  588.0  104.0  168.0   
2  85985  accarje01    2005      1    SFN   NL   28    2.0    0.0    1.0   
3  85986  acevejo01    2005      1    COL   NL   36    8.0    0.0    1.0   
4  85987  adamsmi03    2005      1    MIL   NL   13    0.0    0.0    0.0   

      ...         SB   CS     BB     SO   IBB  HBP   SH   SF  GIDP  \
0     ...        2.0  0.0    7.0    9.0   0.0  1.0  3.0  1.0   2.0   
1     ...       31.0  9.0  117.0  134.0  15.0  6.0  0.0  8.0   7.0   
2     ...        0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   0.0   
3     ...        0.0  0.0    1.0    6.0   0.0  1.0  1.0  0.0   0.0   
4     ...        0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   0.0   

   performance  
0          9.0  
1         87.0  
2          1.0  
3         -5.0  
4          0.0  

[5 rows x 24 column

In [73]:
#now i'll concatenate the average and stddev

recent_non_zero_batting_with_avg=pd.merge(recent_non_zero_batting, yearly_perf_avg, on=['yearID','lgID'])
print recent_non_zero_batting_with_avg.head()
recent_non_zero_batting_with_avg_dev=pd.merge(recent_non_zero_batting_with_avg, yearly_perf_dev, on=['yearID','lgID'])
print recent_non_zero_batting_with_avg_dev.head()

   index   playerID  yearID  stint teamID lgID    G     AB     R      H  \
0  85983  abernbr01    2005      1    MIN   AL   24   67.0   5.0   16.0   
1  85988  adamsru01    2005      1    TOR   AL  139  481.0  68.0  123.0   
2  85990  adkinjo01    2005      1    CHA   AL    5    0.0   0.0    0.0   
3  85991  affelje01    2005      1    KCA   AL   49    0.0   0.0    0.0   
4  85996  allench01    2005      1    TEX   AL   21   53.0   5.0   15.0   

     ...       CS    BB    SO  IBB  HBP   SH   SF  GIDP  performance  \
0    ...      0.0   7.0   9.0  0.0  1.0  3.0  1.0   2.0          9.0   
1    ...      2.0  50.0  57.0  1.0  3.0  3.0  8.0   5.0        104.0   
2    ...      0.0   0.0   0.0  0.0  0.0  0.0  0.0   0.0          0.0   
3    ...      0.0   0.0   0.0  0.0  0.0  0.0  0.0   0.0          0.0   
4    ...      1.0   2.0  13.0  0.0  0.0  1.0  0.0   2.0          3.0   

         avg  
0  22.228385  
1  22.228385  
2  22.228385  
3  22.228385  
4  22.228385  

[5 rows x 25 columns]
   

In [74]:
#I now want to standardize each performance
recent_non_zero_batting_with_avg_dev['stdperf']=pd.Series((recent_non_zero_batting_with_avg_dev.performance-\
                                                              recent_non_zero_batting_with_avg_dev.avg)/\
                                                               recent_non_zero_batting_with_avg_dev.dev)
print recent_non_zero_batting_with_avg_dev.head()

   index   playerID  yearID  stint teamID lgID    G     AB     R      H  \
0  85983  abernbr01    2005      1    MIN   AL   24   67.0   5.0   16.0   
1  85988  adamsru01    2005      1    TOR   AL  139  481.0  68.0  123.0   
2  85990  adkinjo01    2005      1    CHA   AL    5    0.0   0.0    0.0   
3  85991  affelje01    2005      1    KCA   AL   49    0.0   0.0    0.0   
4  85996  allench01    2005      1    TEX   AL   21   53.0   5.0   15.0   

     ...       SO  IBB  HBP   SH   SF  GIDP  performance        avg  \
0    ...      9.0  0.0  1.0  3.0  1.0   2.0          9.0  22.228385   
1    ...     57.0  1.0  3.0  3.0  8.0   5.0        104.0  22.228385   
2    ...      0.0  0.0  0.0  0.0  0.0   0.0          0.0  22.228385   
3    ...      0.0  0.0  0.0  0.0  0.0   0.0          0.0  22.228385   
4    ...     13.0  0.0  0.0  1.0  0.0   2.0          3.0  22.228385   

         dev   stdperf  
0  42.365204 -0.312246  
1  42.365204  1.930160  
2  42.365204 -0.524685  
3  42.365204 -0.524685

In [75]:
#Now I'll join the salary and performance data frames

perf_and_salary=pd.merge(recent_non_zero_batting_with_avg_dev,non_zero_salaries,\
                        on=['yearID','lgID','playerID','teamID'])

print perf_and_salary.head()

   index   playerID  yearID  stint teamID lgID    G     AB     R      H  \
0  85988  adamsru01    2005      1    TOR   AL  139  481.0  68.0  123.0   
1  85991  affelje01    2005      1    KCA   AL   49    0.0   0.0    0.0   
2  85996  allench01    2005      1    TEX   AL   21   53.0   5.0   15.0   
3  85998  almanca01    2005      1    TEX   AL    6    0.0   0.0    0.0   
4  85999  alomasa02    2005      1    TEX   AL   46  128.0  11.0   35.0   

    ...     IBB  HBP   SH   SF  GIDP  performance        avg        dev  \
0   ...     1.0  3.0  3.0  8.0   5.0        104.0  22.228385  42.365204   
1   ...     0.0  0.0  0.0  0.0   0.0          0.0  22.228385  42.365204   
2   ...     0.0  0.0  1.0  0.0   2.0          3.0  22.228385  42.365204   
3   ...     0.0  0.0  0.0  0.0   0.0          0.0  22.228385  42.365204   
4   ...     0.0  1.0  3.0  0.0   3.0         30.0  22.228385  42.365204   

    stdperf   salary  
0  1.930160   316000  
1 -0.524685   950000  
2 -0.453872   380000  
3 -0.5

In [122]:
#I want get ride of all the columns I don't care about, including team and league as I'm only interested in salary vs performance over team
only_perf_and_salary=perf_and_salary.drop(['index','stint','G','AB','R','H','2B','3B','HR','RBI','SB',\
                                           'CS','BB','SO','IBB','HBP','SH','SF','GIDP','performance','avg','dev','teamID','lgID'],1)

print only_perf_and_salary.head()

    playerID  yearID   stdperf   salary
0  adamsru01    2005  1.930160   316000
1  affelje01    2005 -0.524685   950000
2  allench01    2005 -0.453872   380000
3  almanca01    2005 -0.524685  1100000
4  alomasa02    2005  0.183443   550000


In [127]:
 only_perf_and_salary.sort_values(by='yearID',ascending=False).groupby('playerID')

Sources:

http://stackoverflow.com/questions/20995196/python-pandas-counting-and-summing-specific-conditions
http://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html
http://pandas.pydata.org/pandas-docs/version/0.18.1/visualization.html
http://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-object-to-dataframe
https://en.wikipedia.org/wiki/Major_League_Baseball_schedule
http://stackoverflow.com/questions/15891038/pandas-change-data-type-of-columns
http://stackoverflow.com/questions/21231834/creating-a-pandas-dataframe-from-columns-of-other-dataframes-with-similar-indexe
http://stackoverflow.com/questions/13411544/delete-column-from-pandas-dataframe
http://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe